In [1]:
# from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
from keras.layers.merge import *

INPUT_DIM = 4096
TIME_STEPS = 80
# if True, the attention vector is shared across the input_dimensions where the attention is applied.
SINGLE_ATTENTION_VECTOR = True
APPLY_ATTENTION_BEFORE_LSTM = False


def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul


def model_attention_applied_after_lstm():
    inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    lstm_units = 32
    lstm_out = LSTM(lstm_units, return_sequences=True)(inputs)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model


def model_attention_applied_before_lstm():
    inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    attention_mul = attention_3d_block(inputs)
    lstm_units = 32
    attention_mul = LSTM(lstm_units, return_sequences=False)(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model

Using TensorFlow backend.


In [2]:
import numpy as np

np.random.seed(1337)  # for reproducibility
from keras.models import *
from keras.layers import Input, Dense, merge

input_dim = 32


def build_model():
    inputs = Input(shape=(input_dim,))

    # ATTENTION PART STARTS HERE
    attention_probs = Dense(input_dim, activation='softmax', name='attention_vec')(inputs)
    attention_mul = Add([inputs, attention_probs], output_shape=32, name='attention_mul', mode='mul')
    # ATTENTION PART FINISHES HERE

    attention_mul = Dense(64)(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[inputs], output=output)

In [3]:
import keras.backend as K
import numpy as np


def get_activations(model, inputs, print_shape_only=False, layer_name=None):
    # Documentation is available online on Github at the address below.
    # From: https://github.com/philipperemy/keras-visualize-activations
    print('----- activations -----')
    activations = []
    inp = model.input
    if layer_name is None:
        outputs = [layer.output for layer in model.layers]
    else:
        outputs = [layer.output for layer in model.layers if layer.name == layer_name]  # all layer outputs
    funcs = [K.function([inp] + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions
    layer_outputs = [func([inputs, 1.])[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)
        if print_shape_only:
            print(layer_activations.shape)
        else:
            print(layer_activations)
    return activations


def get_data(n, input_dim, attention_column=1):
    """
    Data generation. x is purely random except that it's first value equals the target y.
    In practice, the network should learn that the target = x[attention_column].
    Therefore, most of its attention should be focused on the value addressed by attention_column.
    :param n: the number of samples to retrieve.
    :param input_dim: the number of dimensions of each element in the series.
    :param attention_column: the column linked to the target. Everything else is purely random.
    :return: x: model inputs, y: model targets
    """
    x = np.random.standard_normal(size=(n, input_dim))
    y = np.random.randint(low=0, high=2, size=(n, 1))
    x[:, attention_column] = y[:, 0]
    return x, y


def get_data_recurrent(n, time_steps, input_dim, attention_column=10):
    """
    Data generation. x is purely random except that it's first value equals the target y.
    In practice, the network should learn that the target = x[attention_column].
    Therefore, most of its attention should be focused on the value addressed by attention_column.
    :param n: the number of samples to retrieve.
    :param time_steps: the number of time steps of your series.
    :param input_dim: the number of dimensions of each element in the series.
    :param attention_column: the column linked to the target. Everything else is purely random.
    :return: x: model inputs, y: model targets
    """
    x = np.random.standard_normal(size=(n, time_steps, input_dim))
    y = np.random.randint(low=0, high=2, size=(n, 1))
    x[:, attention_column, :] = np.tile(y[:], (1, input_dim))
    return x, y

In [4]:
from keras.optimizers import RMSprop, Adam

In [5]:
import read_data as rd
num_encoder_tokens = 4096
num_densed_tokens = 1024
latent_dim = 256
num_decoder_tokens = rd.max_dict_length
max_dict_length = rd.max_dict_length
max_sent_length = rd.max_sent_length

In [27]:
encoder_inputs = Input(shape=(80, num_encoder_tokens))
attention_mul = attention_3d_block(encoder_inputs)
# lstm_units = 256
# attention_mul, state_h, state_c = LSTM(lstm_units, return_state=True, implementation=2)(attention_mul)
# output = Dense(1, activation='sigmoid')(attention_mul)
# model = Model(input=[inputs], output=output)

# encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True, implementation=2)
encoder_outputs, state_h, state_c = encoder(attention_mul)
# state_h = attention_3d_block(state_h)
# encoder_outputs, state_h, state_c = encoder(dense_outputs)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, implementation=2)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

# print(model.summary())
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# model = Model([CNN_inputs, decoder_inputs], decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy')
# print(model.summary())

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)




# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

/home/kongou/.local/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_26 (InputLayer)            (None, 80, 4096)      0                                            
____________________________________________________________________________________________________
permute_2 (Permute)              (None, 4096, 80)      0           input_26[0][0]                   
____________________________________________________________________________________________________
reshape_2 (Reshape)              (None, 4096, 80)      0           permute_2[0][0]                  
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 4096, 80)      6480        reshape_2[0][0]                  
___________________________________________________________________________________________

In [6]:
import read_data as rd

In [7]:
def input_generator(batch_size=64, option='all_sentence'):
    if option == 'all_sentence':
        total_movies = len(label.mvlist)
        curr_movie = 0
        curr_sent = 0
        while True:
            feat = np.array([]).reshape(0, 80, 4096)
            sent = np.array([]).reshape(0, max_sent_length, max_dict_length)
            for i in range(batch_size):
                if curr_sent >= len(label.mv2sent(label.mvlist[curr_movie % total_movies])):
                    curr_sent -= len(label.mv2sent(label.mvlist[curr_movie % total_movies]))
                # if curr_sent >= 1:
                #     curr_sent -= 1
                    curr_movie += 1
                feat = np.concatenate((feat, [label.mv2feat(label.mvlist[curr_movie % total_movies])]))
                sent = np.concatenate((sent, [label.sent2onehot(label.mv2sent(label.mvlist[curr_movie % total_movies])[curr_sent])]))
                curr_sent += 1
                
            yield [[feat, sent[:, :-1, :]], sent[:, 1:, :]]
    if option == 'random_each':
        total_movies = len(label.mvlist)
        curr_movie = 0
        while True:
            feat = np.array([]).reshape(0, 80, 4096)
            sent = np.array([]).reshape(0, max_sent_length, max_dict_length)
            for i in range(batch_size):
                # if curr_sent >= len(label.mv2sent(label.mvlist[curr_movie % total_movies])):
                #     curr_sent -= len(label.mv2sent(label.mvlist[curr_movie % total_movies]))
                # if curr_sent >= 1:
                #     curr_sent -= 1
                #     curr_movie += 1
                feat = np.concatenate((feat, [label.mv2feat(label.mvlist[curr_movie % total_movies])]))
                curr_sent = random.randint(0, len(label.mv2sent(label.mvlist[curr_movie % total_movies])) - 1)
                sent = np.concatenate((sent, [label.sent2onehot(label.mv2sent(label.mvlist[curr_movie % total_movies])[curr_sent])]))
                curr_movie += 1
                
            yield [[feat, sent[:, :-1, :]], sent[:, 1:, :]]


In [8]:
# DS = rd.Dataset()
train_DS = rd.Dataset('../hw2_dataset/training_data/feat/', '../hw2_dataset/training_label.json')

In [9]:
label = train_DS.label

In [10]:
gen = input_generator(option='random_each')
model_filename = 'models/1118_att.h5'

In [11]:
print(label)
print(label.mvlist)

['2HpZvL-_fiY_40_44.avi', 'hJFBXHtxKIc_204_209.avi', 'UbmZAe5u5FI_88_98.avi', 'dMH0bHeiRNg_3_14.avi', 'dQmaVQZz7EE_1_18.avi', 'gtixLEvO2Us_0_6.avi', 'UoPU8F9mus8_258_262.avi', 'kWLNZzuo3do_38_42.avi', 'UXs3eq68ZjE_194_200.avi', '4UOVKok7j1U_1_8.avi', '3chNlP5TeO8_0_10.avi', 'HzYtvOYOEoU_21_32.avi', 'WTf5EgVY5uU_76_83.avi', '2YhDTpzxd3c_223_232.avi', '2YhDTpzxd3c_115_120.avi', 'lc9bA-hvqHU_1_6.avi', 'ZYaxg5NZW_g_21_27.avi', 'sBjr8UJOpsQ_181_195.avi', 'j2sOMdilDWU_87_97.avi', 'lFyPUgJCmtU_100_110.avi', '0lh_UWF9ZP4_157_160.avi', 's0hwEUC5emA_127_132.avi', 'PeUHy0A1GF0_62_66.avi', 'ePujnD4qJO0_62_77.avi', 'u9prcUCHlqM_235_241.avi', 'c2MwqFYVE7A_40_45.avi', 'j1Z890_Q3so_131_138.avi', 'k-SWy-sU8cE_5_10.avi', 'i2sRHf9m5KM_28_42.avi', 'YmXCfQm0_CA_109_120.avi', 'qeKX-N1nKiM_52_59.avi', '_O9kWD8nuRU_16_23.avi', 'WPG-BIWOrG4_672_677.avi', 'pRpeEdMmmQ0_65_70.avi', 'KqKUXRHHkCc_101_105.avi', 'Ffv7fhL1EDY_177_184.avi', '8PQiaurIiDM_247_255.avi', 'xxHx6s_DbUo_121_128.avi', '-rvjK0lE3z4_33_43.avi', 

In [12]:
import random
# max_sent_length = rd.max_sent_length
# max_dict_length = rd.max_dict_length

In [ ]:
print(next(gen))

In [41]:
max_score = 0
for i in range(200):
    output_decoded_sequence(encoder_model, decoder_model)
    model.fit_generator(gen, 4, epochs=50, verbose=1)
    score = validation(output_caption_txt(encoder_model, decoder_model))
    if score > max_score:
        model.save_weights(model_filename)

Decoded sentence:  a man is a <end> 
Decoded sentence:  a man is a a a <end> 
Decoded sentence:  a man is a a <end> 
Decoded sentence:  a man is playing <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a man is playing <end> 
Decoded sentence:  a man is slicing a <end> 
Decoded sentence:  a man is a a <end> 
Decoded sentence:  a man is playing <end> 
Epoch 1/50
4/4 [==============================] - 8s - loss: 0.7822     
Epoch 2/50
4/4 [==============================] - 7s - loss: 0.7973     
Epoch 3/50
4/4 [==============================] - 7s - loss: 0.7852     
Epoch 4/50
4/4 [==============================] - 8s - loss: 0.7735     
Epoch 5/50
4/4 [==============================] - 8s - loss: 0.7635     
Epoch 6/50
4/4 [==============================] - 7s - loss: 0.7852     
Epoch 7/50
4/4 [==============================] - 7s - loss: 0.7981     
Epoch 8/50
4/4 [==============================] - 8s - loss: 0.7982  

4/4 [==============================] - 5s - loss: 0.6477     
Epoch 49/50
4/4 [==============================] - 5s - loss: 0.6388     
Epoch 50/50
4/4 [==============================] - 5s - loss: 0.6524     
Originally, average bleu score is 0.24321389901804108
By another method, average bleu score is 0.5508953045342527
Decoded sentence:  a woman is dancing <end> 
Decoded sentence:  a man is adding a into a pot <end> 
Decoded sentence:  a woman is slicing a <end> 
Decoded sentence:  a man is dancing <end> 
Decoded sentence:  a man is playing a guitar <end> 
Decoded sentence:  a man is playing a <end> 
Decoded sentence:  a woman is dancing <end> 
Decoded sentence:  a woman is slicing a potato <end> 
Decoded sentence:  a man is slicing a <end> 
Decoded sentence:  a man is playing a guitar <end> 
Epoch 1/50
4/4 [==============================] - 5s - loss: 0.6644     
Epoch 2/50
4/4 [==============================] - 5s - loss: 0.6233     
Epoch 3/50
4/4 [==============================]

4/4 [==============================] - 6s - loss: 0.5394     
Epoch 43/50
4/4 [==============================] - 6s - loss: 0.5386     
Epoch 44/50
4/4 [==============================] - 6s - loss: 0.6094     
Epoch 45/50
4/4 [==============================] - 6s - loss: 0.5303     
Epoch 46/50
4/4 [==============================] - 6s - loss: 0.6242     
Epoch 47/50
4/4 [==============================] - 6s - loss: 0.6060     
Epoch 48/50
4/4 [==============================] - 6s - loss: 0.5729     
Epoch 49/50
4/4 [==============================] - 6s - loss: 0.5423     
Epoch 50/50
4/4 [==============================] - 6s - loss: 0.5631     
Originally, average bleu score is 0.2537330217831846
By another method, average bleu score is 0.5558919926709869
Decoded sentence:  a woman is dancing <end> 
Decoded sentence:  a man is pouring a pot <end> 
Decoded sentence:  a woman is slicing a piece of a pan <end> 
Decoded sentence:  a man is dancing <end> 
Decoded sentence:  a man is playin

4/4 [==============================] - 9s - loss: 0.4928     
Epoch 37/50
4/4 [==============================] - 9s - loss: 0.5385     
Epoch 38/50
4/4 [==============================] - 9s - loss: 0.5214     
Epoch 39/50
4/4 [==============================] - 9s - loss: 0.5181     
Epoch 40/50
4/4 [==============================] - 9s - loss: 0.5289     
Epoch 41/50
4/4 [==============================] - 9s - loss: 0.4848     
Epoch 42/50
4/4 [==============================] - 9s - loss: 0.5557     
Epoch 43/50
4/4 [==============================] - 9s - loss: 0.5113     
Epoch 44/50
4/4 [==============================] - 9s - loss: 0.5212     
Epoch 45/50
4/4 [==============================] - 9s - loss: 0.5098     
Epoch 46/50
4/4 [==============================] - 9s - loss: 0.5096     
Epoch 47/50
4/4 [==============================] - 9s - loss: 0.5054     
Epoch 48/50
4/4 [==============================] - 9s - loss: 0.5248     
Epoch 49/50
4/4 [==============================] -

4/4 [==============================] - 7s - loss: 0.4817     
Epoch 31/50
4/4 [==============================] - 7s - loss: 0.4769     
Epoch 32/50
4/4 [==============================] - 7s - loss: 0.4798     
Epoch 33/50
4/4 [==============================] - 7s - loss: 0.4964     
Epoch 34/50
4/4 [==============================] - 7s - loss: 0.4285     
Epoch 35/50
4/4 [==============================] - 7s - loss: 0.4949     
Epoch 36/50
4/4 [==============================] - 7s - loss: 0.4947     
Epoch 37/50
4/4 [==============================] - 7s - loss: 0.4283     
Epoch 38/50
4/4 [==============================] - 7s - loss: 0.5404     
Epoch 39/50
4/4 [==============================] - 8s - loss: 0.4545     
Epoch 40/50
4/4 [==============================] - 7s - loss: 0.4714     
Epoch 41/50
4/4 [==============================] - 7s - loss: 0.4750     
Epoch 42/50
4/4 [==============================] - 8s - loss: 0.4635     
Epoch 43/50
4/4 [==============================] -

...

4/4 [==============================] - 6s - loss: 0.1519     
Epoch 49/50
4/4 [==============================] - 6s - loss: 0.1649     
Epoch 50/50
4/4 [==============================] - 6s - loss: 0.1560     
Originally, average bleu score is 0.24509714242188388
By another method, average bleu score is 0.5323902491265254
Decoded sentence:  two women are dancing <end> 
Decoded sentence:  a man is adding wine to a pot <end> 
Decoded sentence:  a woman is putting oil on fish <end> 
Decoded sentence:  a man is dancing <end> 
Decoded sentence:  a boy is playing a guitar <end> 
Decoded sentence:  a woman is dancing <end> 
Decoded sentence:  a woman is exercising <end> 
Decoded sentence:  a woman is slicing a fish <end> 
Decoded sentence:  a man is cracking eggs into a bowl <end> 
Decoded sentence:  a man is playing a guitar <end> 
Epoch 1/50
4/4 [==============================] - 7s - loss: 0.1623     
Epoch 2/50
4/4 [==============================] - 6s - loss: 0.1563     
Epoch 3/50
4/4 [

4/4 [==============================] - 6s - loss: 0.1569     
Epoch 42/50
4/4 [==============================] - 6s - loss: 0.1516     
Epoch 43/50
4/4 [==============================] - 6s - loss: 0.1524     
Epoch 44/50
4/4 [==============================] - 6s - loss: 0.1523     
Epoch 45/50
4/4 [==============================] - 6s - loss: 0.1677     
Epoch 46/50
4/4 [==============================] - 6s - loss: 0.1478     
Epoch 47/50
4/4 [==============================] - 6s - loss: 0.1712     
Epoch 48/50
4/4 [==============================] - 6s - loss: 0.1492     
Epoch 49/50
4/4 [==============================] - 6s - loss: 0.1612     
Epoch 50/50
4/4 [==============================] - 6s - loss: 0.1542     
Originally, average bleu score is 0.24387734961347074
By another method, average bleu score is 0.527909124563421
Decoded sentence:  two women are dancing in a circle out of a room <end> 
Decoded sentence:  a man is pouring wine into a pot <end> 
Decoded sentence:  a woman

4/4 [==============================] - 6s - loss: 0.1543     
Epoch 34/50
4/4 [==============================] - 6s - loss: 0.1493     
Epoch 35/50
4/4 [==============================] - 6s - loss: 0.1540     
Epoch 36/50
4/4 [==============================] - 6s - loss: 0.1589     
Epoch 37/50
4/4 [==============================] - 6s - loss: 0.1649     
Epoch 38/50
4/4 [==============================] - 6s - loss: 0.1490     
Epoch 39/50
4/4 [==============================] - 6s - loss: 0.1518     
Epoch 40/50
4/4 [==============================] - 6s - loss: 0.1451     
Epoch 41/50
4/4 [==============================] - 6s - loss: 0.1614     
Epoch 42/50
4/4 [==============================] - 6s - loss: 0.1486     
Epoch 43/50
4/4 [==============================] - 6s - loss: 0.1438     
Epoch 44/50
4/4 [==============================] - 6s - loss: 0.1601     
Epoch 45/50
4/4 [==============================] - 6s - loss: 0.1518     
Epoch 46/50
4/4 [==============================] -

4/4 [==============================] - 6s - loss: 0.1535     
Epoch 26/50
4/4 [==============================] - 6s - loss: 0.1487     
Epoch 27/50
4/4 [==============================] - 6s - loss: 0.1468     
Epoch 28/50
4/4 [==============================] - 6s - loss: 0.1469     
Epoch 29/50
4/4 [==============================] - 6s - loss: 0.1362     
Epoch 30/50
4/4 [==============================] - 6s - loss: 0.1424     
Epoch 31/50
4/4 [==============================] - 6s - loss: 0.1452     
Epoch 32/50
4/4 [==============================] - 6s - loss: 0.1454     
Epoch 33/50
4/4 [==============================] - 6s - loss: 0.1618     
Epoch 34/50
4/4 [==============================] - 6s - loss: 0.1520     
Epoch 35/50
4/4 [==============================] - 6s - loss: 0.1471     
Epoch 36/50
4/4 [==============================] - 6s - loss: 0.1532     
Epoch 37/50
4/4 [==============================] - 6s - loss: 0.1552     
Epoch 38/50
4/4 [==============================] -

KeyboardInterrupt: 

In [ ]:
print(next(gen))

In [29]:
test_DS = rd.Dataset('../hw2_dataset/testing_data/feat/', '../hw2_dataset/testing_label.json')

In [30]:
import bleu_eval as be

In [31]:
def validation(result):
    return be.val_BLEU(result)

In [42]:
def output_caption_txt(e, d):
    movies = test_DS.label.mvlist
    # with open('caption.txt', 'w') as f:

    result = {}
    for index, movie in enumerate(movies):
#         print(index, ':')
        input_seq = test_DS.label.mv2feat(movie)
        decoded_sentence = decode_sequence(input_seq.reshape(1, 80, 4096), e, d)
        print('Video Name: ', movie)
        print('Decoded Sentence: ', decoded_sentence)
        tmp = (decoded_sentence[:-7] + '.').capitalize()
        result[movie] = tmp
#         print(movie + ',' + tmp)
        # for sent in label.mv2sent(movie):
        #     print(sent)
        # f.write(movie + ',' + tmp + '\n')
    return result

In [33]:
def output_decoded_sequence(e, d, test=10):
    movies = label.mvlist
    for index, movie in enumerate(movies[:test]):
        input_seq = label.mv2feat(movie)
        decoded_sentence = decode_sequence(input_seq.reshape(1, 80, 4096), e, d)
        print('Decoded sentence: ', decoded_sentence)

In [34]:
def decode_sequence(input_seq, encoder_model, decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, label.word2int('<start>')] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
#         print(output_tokens[0, -1, 73])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = label.int2word(int(sampled_token_index))
        decoded_sentence += sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_word == '<end>':
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
        decoded_sentence += ' ' 

    return decoded_sentence

In [43]:
validation(output_caption_txt(encoder_model, decoder_model))

Video Name:  mmSQTI6gMNQ_120_128.avi
Decoded Sentence:  the ladies danced in front of the ocean <end> 
Video Name:  pW9DFPqoIsI_26_50.avi
Decoded Sentence:  a woman is chopping a vegetable <end> 
Video Name:  ZbtpcGi2DWY_161_170.avi
Decoded Sentence:  a man is cleaning itself with water <end> 
Video Name:  MTjrZthHwJQ_2_11.avi
Decoded Sentence:  a woman is holding a baby monkey <end> 
Video Name:  3qqEKTPxLNs_1_15.avi
Decoded Sentence:  a baby is sitting on a sofa is trying to get out of the cat's nose and the man <end> 
Video Name:  RZL9irxnhZ0_34_40.avi
Decoded Sentence:  a man is talking about a gray you tube on a card using a <unk> <unk> <end> 
Video Name:  6JnGBs88sL0_4_10.avi
Decoded Sentence:  a man is applying makeup <end> 
Video Name:  77iDIp40m9E_126_131.avi
Decoded Sentence:  a man is trying to catch a dog <end> 
Video Name:  lo4KcsBN--A_0_10.avi
Decoded Sentence:  a man is driving a car <end> 
Video Name:  PeUHy0A1GF0_114_121.avi
Decoded Sentence:  a woman is slicing some f

Video Name:  0lh_UWF9ZP4_62_69.avi
Decoded Sentence:  a woman is cracking an egg <end> 
Video Name:  BAf3LXFUaGs_28_38.avi
Decoded Sentence:  a man is playing a saxophone <end> 
Video Name:  WTf5EgVY5uU_124_128.avi
Decoded Sentence:  a woman is cooking some egg <end> 
Video Name:  n016q1w8Q30_2_11.avi
Decoded Sentence:  a person is chopping an onion <end> 
Video Name:  RSx5G0_xH48_12_17.avi
Decoded Sentence:  a cat is playing with a ball <end> 
Video Name:  Je3V7U5Ctj4_569_576.avi
Decoded Sentence:  a man is spreading butter on a tortilla <end> 
Video Name:  UXs3eq68ZjE_250_255.avi
Decoded Sentence:  someone is pouring rice into a pot <end> 
Video Name:  YmXCfQm0_CA_277_284.avi
Decoded Sentence:  a man is climbing a rock <end> 
Video Name:  1Sp2__RCT0c_11_15.avi
Decoded Sentence:  a man is riding a motorcycle <end> 
Video Name:  j2Dhf-xFUxU_13_20.avi
Decoded Sentence:  a woman is peeling a potato <end> 
Originally, average bleu score is 0.2509711569571993
By another method, average ble

0.540359673512018